In [1]:
import asyncio
import itertools
import json
import logging
import math
import multiprocessing
import random
import sys
import time
import os
from threading import Thread
from urllib.parse import unquote,quote
import aiohttp
import pandas as pd
import pymongo
import requests
from lxml import etree
from concurrent.futures import ThreadPoolExecutor, wait, ALL_COMPLETED
import hashlib
import csv


In [2]:
client = pymongo.MongoClient(host='localhost', port=27017)
db = client['traffic']
metro_station = db['metro_station_v2']
metro_line = db['metro_line_v2']





In [3]:
next(metro_station.find())

{'_id': 'BV10013454',
 'su': '1',
 'udsu': '1;1',
 'n': '苹果园',
 'sid': '110100023339034',
 'r': [{'lid': '900000069871', 'ln': 'S1线'},
  {'lid': '110100023339', 'ln': '6号线'}],
 'udsi': '110100023339034;110100023340035',
 't': '1',
 'si': '110100023339034',
 'sl': '116.178945,39.925686',
 'udli': '110100023339;110100023340',
 'lg': '0',
 'sp': 'PingGuoYuan',
 'lnt': 116.178945,
 'lat': 39.925686,
 'cn': '北京市'}

In [13]:
fs= open('station.csv','w',encoding='utf_8_sig',newline='')
csv_w = csv.writer(fs)
col = ['id','name','lng','lat','cityName','lineNames','lineID']
csv_w.writerow(col)
for item in  metro_station.find():
    print(item['r'],item['_id'])
    ln = map(lambda x:x['ln'],item['r'])

    ln = '|'.join(ln)
    lid = list(map(lambda x:x['lid'],item['r']))
    lid = '|'.join(lid)
    row = [item['_id'],item['n'],item['lng'], item['lat'], item['cn'],ln,lid]
    csv_w.writerow(row)
    

[{'lid': '900000069871', 'ln': 'S1线'}, {'lid': '110100023339', 'ln': '6号线'}] BV10013454
[{'lid': '900000069871', 'ln': 'S1线'}, {'lid': '110100023339', 'ln': '6号线'}, {'lid': '900000159716', 'ln': '11号线'}] BV11494889
[{'lid': '900000069871', 'ln': 'S1线'}] BV10813547
[{'lid': '900000069871', 'ln': 'S1线'}] BV10813541
[{'lid': '900000069871', 'ln': 'S1线'}] BV10813543
[{'lid': '900000069871', 'ln': 'S1线'}] BV10813544
[{'lid': '900000069871', 'ln': 'S1线'}] BV10813542
[{'lid': '900000069871', 'ln': 'S1线'}] BV10813546
[{'lid': '110100023110', 'ln': '1号线八通线'}] BV10000295
[{'lid': '110100023110', 'ln': '1号线八通线'}] BV10013455
[{'lid': '110100023110', 'ln': '1号线八通线'}] BV10000302
[{'lid': '110100023110', 'ln': '1号线八通线'}] BV10013456
[{'lid': '110100023110', 'ln': '1号线八通线'}] BV10013457
[{'lid': '110100023110', 'ln': '1号线八通线'}] BV10013458
[{'lid': '110100023110', 'ln': '1号线八通线'}, {'lid': '110100023282', 'ln': '10号线'}] BV10000306
[{'lid': '110100023110', 'ln': '1号线八通线'}, {'lid': '110100023116', 'ln': '9号

In [17]:
fs= open('metro_line.csv','w',encoding='utf_8_sig',newline='')
csv_w = csv.writer(fs)
col = ['id','name','cityName','st_station','ed_station']
csv_w.writerow(col)
for item in  metro_line.find():
    st = item['st'][0]['n']
    ed = item['st'][-1]['n']
    
    row = [item['_id'],item['ln'],item['cn'],st, ed]
    csv_w.writerow(row)
fs.close()    

In [3]:
sz_record =  db['sz_metro_records']
ag_l = [{
    '$group': {
        '_id': {
            'card_no': '$card_no'
        },
        'ids': {
            '$addToSet': '$_id'
        }
    }
}, {
    '$unwind': '$ids'
}, {
    '$group': {
        '_id': '$_id',
        'count_id': {
            '$sum': 1
        }
    }
}]
m_idl = []
for item in sz_record.aggregate(ag_l,allowDiskUse=True):
    if item['count_id'] > 1:
        m_idl.append({'card_no':item['_id']['card_no'],'count': item['count_id']})
   


In [50]:
fs= open('person.csv','w',encoding='utf_8_sig',newline='')
csv_w = csv.writer(fs)
col = ['id','count']
csv_w.writerow(col)

for item in m_idl:
    try:
        row = [item['card_no'],item['count']]
    except Exception:
        print(item)
    csv_w.writerow(row)
fs.close()

In [4]:
m_idl[1:10]

[{'card_no': 'AEAAAAAFB', 'count': 2},
 {'card_no': 'AEAAAACFF', 'count': 2},
 {'card_no': 'AEAAAACHG', 'count': 2},
 {'card_no': 'AEAAAAEBF', 'count': 2},
 {'card_no': 'AEAAAAHFB', 'count': 2},
 {'card_no': 'AEAAAAHIC', 'count': 2},
 {'card_no': 'AEAAACAHB', 'count': 2},
 {'card_no': 'AEAAACBHI', 'count': 4},
 {'card_no': 'AEAAACCBI', 'count': 2}]

In [7]:
fs = open('route_record.csv', 'w', encoding='utf_8_sig', newline='')
csv_w = csv.writer(fs)
col = [
    'id', 'name', 'card_no', 'st_station', 'ed_station', 'st_time', 'ed_time','line_name'
]
ans = []
csv_w.writerow(col)
def process(t_item):
    record_li = []
    for item in sz_record.find({'card_no': t_item['card_no']}):
        record_li.append(item)
    record_sort_li = sorted(record_li,
                            key=lambda x: x['deal_date'],
                            reverse=False)
    if t_item['count'] % 2 != 0:
        record_sort_li = record_sort_li[:-1]
    for i in range(0, len(record_sort_li), 2):
        name = record_sort_li[i]['station'] + '->' + record_sort_li[i + 1]['station']
        _id = record_sort_li[i]['_id']
        card_no = record_sort_li[i]['card_no']
        st_station = record_sort_li[i]['station']
        ed_station = record_sort_li[i + 1]['station']
        st_time = record_sort_li[i]['deal_date']
        ed_time = record_sort_li[i + 1]['deal_date'] 
        line_name = record_sort_li[i]['company_name'] + '->' + record_sort_li[i + 1]['company_name']
        row = [_id,name,card_no,st_station,ed_station,st_time,ed_time,line_name]
        csv_w.writerow(row)
#         print(row)
#         ans.append(row)
        
def run_record():
    with ThreadPoolExecutor(max_workers=500) as executor:
        future_tasks = []
        for t_item in m_idl:
             future_tasks.append(
                    executor.submit(process, t_item))
        wait(future_tasks, return_when=ALL_COMPLETED)

# run_record()
# csv_w.writerows(ans)
fs.close()


['b2f3a6266b1811ec94acf8e4e3b06c48', '田贝->大剧院站', 'AEAAAAHIC', '田贝', '大剧院站', '2018-09-01 10:32:34', '2018-09-01 10:45:15', '地铁三号线->地铁一号线']
['b1ce11f86b1811ec8653f8e4e3b06c48', '向西村->少年宫', 'AEAAAAEBF', '向西村', '少年宫', '2018-09-01 09:35:25', '2018-09-01 10:08:21', '地铁九号线->地铁四号线']
['afa4f8016b1811ec8933f8e4e3b06c48', '泥岗->黄木岗', 'AEAAAAAFB', '泥岗', '黄木岗', '2018-09-01 07:25:16', '2018-09-01 07:41:38', '地铁九号线->地铁七号线']
['afd2beca6b1811ec8f71f8e4e3b06c48', '草埔->少年宫', 'AEAAAACHG', '草埔', '少年宫', '2018-09-01 07:47:02', '2018-09-01 08:17:53', '地铁三号线->地铁三号线']
['b042dd916b1811eca1d9f8e4e3b06c48', '翠竹->少年宫', 'AEAAAACFF', '翠竹', '少年宫', '2018-09-01 08:26:22', '2018-09-01 08:49:23', '地铁三号线->地铁四号线']
['b05cd7e36b1811eca0a0f8e4e3b06c48', '桃源村->留仙洞', 'AEAAAAAAG', '桃源村', '留仙洞', '2018-09-01 08:35:05', '2018-09-01 08:18:04', '地铁七号线->地铁五号线']
['afa6cd1e6b1811ecbca9f8e4e3b06c48', '莲花村->华新', 'AEAAACAHB', '莲花村', '华新', '2018-09-01 07:32:55', '2018-09-01 07:26:29', '地铁三号线->地铁三号线']
['aff4ec7e6b1811ec90f3f8e4e3b06c48', '太安->

['afe3d5996b1811eca0d0f8e4e3b06c48', '世界之窗站->红树湾', 'AEAAJIBAE', '世界之窗站', '红树湾', '2018-09-01 07:51:49', '2018-09-01 07:59:26', '地铁一号线->地铁二号线']
['b2c6a32c6b1811ec8b4bf8e4e3b06c48', '珠光->后海', 'AEAAJHIGG', '珠光', '后海', '2018-09-01 10:21:54', '2018-09-01 10:54:53', '地铁七号线->地铁二号线']
['b21d6a826b1811ecb557f8e4e3b06c48', '大新->香蜜', 'AEAAJHBHH', '大新', '香蜜', '2018-09-01 09:57:02', '2018-09-01 10:27:11', '地铁一号线->地铁二号线']
['afe55c676b1811ec962bf8e4e3b06c48', '莲花西->桃源村', 'AEAAJHDHE', '莲花西', '桃源村', '2018-09-01 07:51:15', '2018-09-01 08:15:01', '地铁二号线->地铁七号线']
['b0cc0b1e6b1811ecb458f8e4e3b06c48', '怡景->怡景', 'AEAAJIIEC', '怡景', '怡景', '2018-09-01 08:49:12', '2018-09-01 09:00:25', '地铁五号线->地铁五号线']
['b13b95ae6b1811ecb431f8e4e3b06c48', '大芬->长龙', 'AEAAJIIEC', '大芬', '长龙', '2018-09-01 09:25:27', '2018-09-01 08:30:56', '地铁三号线->地铁五号线']
['af7498fa6b1811ecb412f8e4e3b06c48', '黄贝岭->双龙', 'AEAAJJGEH', '黄贝岭', '双龙', '2018-09-01 06:59:49', '2018-09-01 08:06:29', '地铁五号线->地铁三号线']
['b43f174d6b1811ec9c96f8e4e3b06c48', '华新->通新岭', 

['b32671436b1811ecb218f8e4e3b06c48', '西丽湖->香蜜', 'AEAJBCHIG', '西丽湖', '香蜜', '2018-09-01 10:47:37', '2018-09-01 11:15:43', '地铁七号线->地铁二号线']
['b4dfecb96b1811ec9525f8e4e3b06c48', '香蜜->西丽湖', 'AEAJBCHIG', '香蜜', '西丽湖', '2018-09-01 09:12:02', '2018-09-01 09:44:41', '地铁二号线->地铁七号线']
['b12d65296b1811ec98e0f8e4e3b06c48', '丹竹头->华新', 'AEAJBCIHE', '丹竹头', '华新', '2018-09-01 09:29:19', '2018-09-01 10:01:09', '地铁三号线->地铁三号线']
['b03d5ef66b1811ecb7e3f8e4e3b06c48', '杨美->黄贝岭', 'AEAJBDABC', '杨美', '黄贝岭', '2018-09-01 08:19:32', '2018-09-01 08:45:38', '地铁五号线->地铁五号线']
['afe18bae6b1811eca139f8e4e3b06c48', '田贝->晒布', 'AEAJBDABD', '田贝', '晒布', '2018-09-01 07:52:53', '2018-09-01 08:00:37', '地铁三号线->地铁三号线']
['b2c4f6946b1811ecacbcf8e4e3b06c48', '侨香->益田', 'AEAJBDGEB', '侨香', '益田', '2018-09-01 10:30:33', '2018-09-01 10:10:17', '地铁二号线->地铁三号线']
['b21505d16b1811ecbac7f8e4e3b06c48', '珠光->沙尾', 'AEAJBEHGI', '珠光', '沙尾', '2018-09-01 09:58:30', '2018-09-01 10:21:03', '地铁七号线->地铁七号线']
['afbd62296b1811ec8a06f8e4e3b06c48', '沙尾->水湾', 'AEAJBE

['afa34a8f6b1811ecbf19f8e4e3b06c48', '福永站->吉祥', 'AEAJDACAB', '福永站', '吉祥', '2018-09-01 07:27:22', '2018-09-01 09:23:38', '地铁十一号线->地铁三号线']['b41f0dd86b1811ec9613f8e4e3b06c48', '宝体->灵芝', 'AEAJDDJHG', '宝体', '灵芝', '2018-09-01 08:54:26', '2018-09-01 09:09:03', '地铁一号线->地铁五号线']

['b30d21806b1811ec86acf8e4e3b06c48', '下梅林->侨香', 'AEAJDEIJI', '下梅林', '侨香', '2018-09-01 10:35:28', '2018-09-01 11:00:03', '地铁九号线->地铁二号线']
['b10b36f36b1811ecaccdf8e4e3b06c48', '福田口岸->皇岗村', 'AEAJDDHFC', '福田口岸', '皇岗村', '2018-09-01 08:25:00', '2018-09-01 08:04:24', '地铁四号线->地铁七号线']
['af5f61096b1811ecaf29f8e4e3b06c48', '翻身->深圳北', 'AEAJDFFDF', '翻身', '深圳北', '2018-09-01 06:22:38', '2018-09-01 07:02:24', '地铁五号线->地铁五号线']
['af64bc446b1811ec9459f8e4e3b06c48', '侨香->长龙', 'AEAJDBEDG', '侨香', '长龙', '2018-09-01 06:31:47', '2018-09-01 07:21:24', '地铁二号线->地铁五号线']
['b328bb8c6b1811ec83cef8e4e3b06c48', '黄贝岭->百鸽笼', 'AEAJDGBCE', '黄贝岭', '百鸽笼', '2018-09-01 10:45:04', '2018-09-01 10:59:01', '地铁五号线->地铁五号线']
['b02cee896b1811ecb74cf8e4e3b06c48', '西丽湖->福民

['afeab3436b1811ec9fd8f8e4e3b06c48', '怡景->大运', 'AEAJFCHGD', '怡景', '大运', '2018-09-01 07:44:13', '2018-09-01 08:31:21', '地铁五号线->地铁三号线']
['afc21cda6b1811ec82b1f8e4e3b06c48', '洪湖->莲花村', 'AEAJFDHFC', '洪湖', '莲花村', '2018-09-01 07:38:05', '2018-09-01 08:08:21', '地铁七号线->地铁三号线']
['b15908dd6b1811ec8888f8e4e3b06c48', '洪湖->赤尾', 'AEAJEIGDB', '洪湖', '赤尾', '2018-09-01 09:17:58', '2018-09-01 09:41:21', '地铁七号线->地铁七号线']
['b1a7ec076b1811ec8bdbf8e4e3b06c48', '罗湖站->大剧院站', 'AEAJFDJHB', '罗湖站', '大剧院站', '2018-09-01 09:40:14', '2018-09-01 09:30:23', '地铁一号线->地铁一号线']
['af9407c96b1811ec9776f8e4e3b06c48', '大新->留仙洞', 'AEAJFBJJC', '大新', '留仙洞', '2018-09-01 07:16:45', '2018-09-01 07:52:49', '地铁一号线->地铁五号线']
['af89f5b76b1811ec8572f8e4e3b06c48', '草埔->草埔', 'AEAJFFEIA', '草埔', '草埔', '2018-09-01 07:15:47', '2018-09-01 10:29:05', '地铁三号线->地铁三号线']
['b29f1dc46b1811ecbf94f8e4e3b06c48', '大芬->太安', 'AEAJFFEAA', '大芬', '太安', '2018-09-01 10:23:28', '2018-09-01 09:59:50', '地铁三号线->地铁五号线']
['aff86f536b1811ec9ae4f8e4e3b06c48', '莲花村->通新岭', 'AE

['afa8a1146b1811ecac7bf8e4e3b06c48', '莲花村->晒布', 'AEAJGJIFI', '莲花村', '晒布', '2018-09-01 07:26:16', '2018-09-01 07:46:17', '地铁三号线->地铁三号线']
['b3144ee36b1811ecb4c1f8e4e3b06c48', '莲花北->老街', 'AEAJHAGJH', '莲花北', '老街', '2018-09-01 10:35:42', '2018-09-01 10:59:14', '地铁四号线->地铁三号线']
['afe92d1b6b1811ec8aadf8e4e3b06c48', '草埔->莲花村', 'AEAJGJHFJ', '草埔', '莲花村', '2018-09-01 07:53:37', '2018-09-01 08:23:35', '地铁三号线->地铁三号线']
['b23ba0846b1811ecbfb9f8e4e3b06c48', '皇岗村->益田', 'AEAJHAIFH', '皇岗村', '益田', '2018-09-01 09:43:40', '2018-09-01 09:59:50', '地铁七号线->地铁三号线']
['b2abeed66b1811eca1a2f8e4e3b06c48', '桃园->国贸站', 'AEAJHBDFA', '桃园', '国贸站', '2018-09-01 10:32:05', '2018-09-01 11:13:48', '地铁一号线->地铁一号线']
['aff5fdde6b1811ec87b9f8e4e3b06c48', '红山->莲花北', 'AEAJHAFAE', '红山', '莲花北', '2018-09-01 07:48:34', '2018-09-01 08:03:08', '地铁四号线->地铁四号线']
['b08208216b1811ec8adff8e4e3b06c48', '福民->福田口岸', 'AEAJHAICA', '福民', '福田口岸', '2018-09-01 08:00:08', '2018-09-01 08:06:13', '地铁四号线->地铁四号线']
['b3088d826b1811ecb129f8e4e3b06c48', '会展中心站->福

['af694e746b1811ec9ee5f8e4e3b06c48', '侨城东站->晒布', 'AEAJIDACJ', '侨城东站', '晒布', '2018-09-01 06:32:02', '2018-09-01 07:10:04', '地铁一号线->地铁三号线']
['b03293a96b1811ecb318f8e4e3b06c48', '科学馆站->石厦', 'AEAJIBCCC', '科学馆站', '石厦', '2018-09-01 08:29:21', '2018-09-01 08:54:53', '地铁一号线->地铁三号线']
['b1ad1c126b1811ecb0f6f8e4e3b06c48', '翻身->东角头', 'AEAJIDFJI', '翻身', '东角头', '2018-09-01 09:35:51', '2018-09-01 08:54:20', '地铁五号线->地铁二号线']
['b151b5da6b1811ecbb29f8e4e3b06c48', '香蜜->福田口岸', 'AEAJICIHH', '香蜜', '福田口岸', '2018-09-01 09:21:53', '2018-09-01 09:53:10', '地铁二号线->地铁四号线']
['af9dccff6b1811ec947cf8e4e3b06c48', '坂田->民治', 'AEAJIDGFA', '坂田', '民治', '2018-09-01 07:22:42', '2018-09-01 07:36:47', '地铁五号线->地铁五号线']
['b088e6246b1811ecbdbdf8e4e3b06c48', '香梅北->?I岭', 'AEAJIDHEH', '香梅北', '?I岭', '2018-09-01 08:37:10', '2018-09-01 09:29:22', '地铁二号线->地铁九号线']
['afbd13eb6b1811ecb9d4f8e4e3b06c48', '香梅->黄贝岭', 'AEAJIECIC', '香梅', '黄贝岭', '2018-09-01 07:32:27', '2018-09-01 08:05:05', '地铁九号线->地铁五号线']
['b34ea7a76b1811ecb45df8e4e3b06c48', '黄贝岭-

['b24bcd9a6b1811ec9fbef8e4e3b06c48', '华强南->塘尾站', 'AEAJJGFJH', '华强南', '塘尾站', '2018-09-01 09:55:38', '2018-09-01 11:06:38', '地铁七号线->地铁十一号线']
['b0af824a6b1811ec8e8ff8e4e3b06c48', '大新->后瑞', 'AEAJJICBB', '大新', '后瑞', '2018-09-01 08:50:58', '2018-09-01 09:25:50', '地铁一号线->地铁一号线']
['af8abad36b1811ec8f27f8e4e3b06c48', '香梅->文锦', 'AEAJJHBGE', '香梅', '文锦', '2018-09-01 07:11:22', '2018-09-01 07:48:56', '地铁九号线->地铁九号线']
['b0243bc56b1811ecb236f8e4e3b06c48', '宝体->海月', 'AEAJJICBE', '宝体', '海月', '2018-09-01 07:57:16', '2018-09-01 08:40:29', '地铁一号线->地铁二号线']
['af5cf01c6b1811eca2dcf8e4e3b06c48', '龙华->龙华', 'AEIJJJJAA', '龙华', '龙华', '2018-09-01 04:19:54', '2018-09-01 04:20:25', '地铁四号线->地铁四号线']
['b312a01f6b1811eca264f8e4e3b06c48', '福田->福田', 'AEJACAGCA', '福田', '福田', '2018-09-01 10:42:38', '2018-09-01 10:53:26', '地铁三号线->地铁三号线']
['af71b2c76b1811ecb67af8e4e3b06c48', '沙尾->翠竹', 'AEJJECJFJ', '沙尾', '翠竹', '2018-09-01 06:44:43', '2018-09-01 07:20:13', '地铁七号线->地铁三号线']
['b013e68d6b1811ecb3f9f8e4e3b06c48', '长龙->双龙', 'AEJACICHD

['b109b03f6b1811ec9a18f8e4e3b06c48', '益田->燕南', 'AHJAJJBIG', '益田', '燕南', '2018-09-01 09:13:20', '2018-09-01 09:34:37', '地铁三号线->地铁二号线']
['b361744a6b1811ec9e67f8e4e3b06c48', '燕南->益田', 'AHJAJJBIG', '燕南', '益田', '2018-09-01 11:04:29', '2018-09-01 11:28:11', '地铁二号线->地铁三号线']
['b3331b4d6b1811ec814af8e4e3b06c48', '大芬->翠竹', 'AHJAJIAGJ', '大芬', '翠竹', '2018-09-01 11:00:18', '2018-09-01 11:19:32', '地铁三号线->地铁三号线']
['afedc0cd6b1811ec945df8e4e3b06c48', '鲤鱼门->香蜜湖站', 'AHJAJJEID', '鲤鱼门', '香蜜湖站', '2018-09-01 07:49:28', '2018-09-01 08:19:12', '地铁一号线->地铁一号线']
['b085b1b86b1811ecaa7cf8e4e3b06c48', '深康->科学馆站', 'AHJAJJBJE', '深康', '科学馆站', '2018-09-01 08:40:42', '2018-09-01 09:19:14', '地铁二号线->地铁一号线']
['b17da7ff6b1811ec885ef8e4e3b06c48', '下水径->坂田', 'AHJAJIHGD', '下水径', '坂田', '2018-09-01 09:23:05', '2018-09-01 09:37:44', '地铁五号线->地铁五号线']
['af89d0266b1811eca810f8e4e3b06c48', '草埔->海月', 'AHJAJIHJD', '草埔', '海月', '2018-09-01 07:14:26', '2018-09-01 08:08:54', '地铁三号线->地铁二号线']
['afaf581c6b1811ec984cf8e4e3b06c48', '大新->通新岭', 'A

['b0e056cb6b1811ec9b9af8e4e3b06c48', '马鞍山->吉祥', 'AHJJAJIIG', '马鞍山', '吉祥', '2018-09-01 08:50:14', '2018-09-01 10:46:35', '地铁十一号线->地铁三号线']
['b0506baa6b1811eca1c3f8e4e3b06c48', '丹竹头->福田', 'AHJJBAECC', '丹竹头', '福田', '2018-09-01 08:32:40', '2018-09-01 09:11:40', '地铁三号线->地铁三号线']
['af8e14a36b1811ecac13f8e4e3b06c48', '清湖->红山', 'AHJJBCDCF', '清湖', '红山', '2018-09-01 07:06:50', '2018-09-01 07:20:22', '地铁四号线->地铁四号线']
['b1d11ed26b1811ec9db4f8e4e3b06c48', '新安->深圳北', 'AHJJBCIIC', '新安', '深圳北', '2018-09-01 09:34:33', '2018-09-01 10:18:57', '地铁一号线->地铁五号线']
['b1d11f286b1811ec9be4f8e4e3b06c48', '碧海湾站->后亭', 'AHJJBCGEA', '碧海湾站', '后亭', '2018-09-01 09:30:12', '2018-09-01 10:02:12', '地铁十一号线->地铁十一号线']
['afc159ae6b1811ecb25ff8e4e3b06c48', '宝体->桃园', 'AHJJBBDDC', '宝体', '桃园', '2018-09-01 07:31:59', '2018-09-01 07:53:10', '地铁一号线->地铁一号线']
['af6183fe6b1811ec8a7df8e4e3b06c48', '洪浪北->深圳北', 'AHJJBCCCA', '洪浪北', '深圳北', '2018-09-01 06:30:02', '2018-09-01 07:03:13', '地铁五号线->地铁五号线']


In [35]:
df = pd.read_html('https://www.bjsubway.com/station/zjgls/')
df

[   1号线路网车站站间距信息                                                   
            起始站名   目的站名 距离（米） Unnamed: 3_level_1 Unnamed: 4_level_1
 0            古城  八角游乐园  1921                NaN                NaN
 1         八角游乐园    八宝山  1953                NaN                NaN
 2           八宝山    玉泉路  1479                NaN                NaN
 3           玉泉路    五棵松  1810                NaN                NaN
 4           五棵松    万寿路  1778                NaN                NaN
 5           万寿路    公主坟  1313                NaN                NaN
 6           公主坟  军事博物馆  1172                NaN                NaN
 7         军事博物馆    木樨地  1166                NaN                NaN
 8           木樨地   南礼士路  1291                NaN                NaN
 9          南礼士路    复兴门   424                NaN                NaN
 10          复兴门     西单  1596                NaN                NaN
 11           西单   天安门西  1217                NaN                NaN
 12         天安门西   天安门东   925                NaN

In [19]:
[{
    '$group': {
        '_id': {
            'card_no': '$card_no'
        },
        'ids': {
            '$addToSet': '$_ id'
        }
    }
}, {
    '$unwind': '$ids'
}, {
    '$group': {
        '_id': '$_id',
        'count_id': {
            '$sum': 1
        }
    }
}]

[{'$group': {'_id': {'card_no': '$card_no'}, 'ids': {'$addToSet': '$_ id'}}},
 {'$unwind': '$ids'},
 {'$group': {'_id': '$_id', 'count_id': {'$sum': 1}}}]

In [10]:
df = pd.read_html('https://www.bjsubway.com/station/smcsj/')


1号线八通线首末车时刻表                                                        
           车站名称 往环球度假区方向                            往古城方向              
           车站名称     首车时间   末车时间                      首车时间   末车时间       
           车站名称     首车时间     全程         半程四惠东        首车时间   末车时间       
           车站名称     首车时间  周日至周四  周五、周六  周日至周四 周五、周六  首车时间  周日至周四  周五、周六
0           苹果园       ――     ――     ――     ――    ――    ――     ――     ――
1            古城     4:57  22:51  23:52  23:33  0:34    ――     ――     ――
2         八角游乐园     5:00  22:54  23:55  23:36  0:37  5:31   0:17   1:21
3           八宝山     5:03  22:57  23:58  23:39  0:40  5:28   0:14   1:18
4           玉泉路     5:06  22:59   0:00  23:41  0:42  5:25   0:12   1:16
5           五棵松     5:08  23:02   0:03  23:44  0:45  5:23   0:09   1:13
6           万寿路     5:11  23:05   0:06  23:47  0:48  5:20   0:06   1:10
7           公主坟     5:13  23:07   0:08  23:49  0:50  5:17   0:04   1:08
8         军事博物馆     5:16  23:10   0:11  23:52  0:53  5:15   0:01   1:05
9           木樨地     5:18  23:12   0:13  23:54  0:55  5:13  23:59   1:03
10         南礼士路     5:20  23:14   0:15  23:56  0:57  5:11  23:57   1:01
11          复兴门     5:13  23:16   0:17  23:58  0:59  5:08  23:55   0:59
12           西单     5:17  23:19   0:20   0:01  1:02  5:17  23:53   0:57
13         天安门西     5:19  23:21   0:22   0:03  1:04  5:14  23:50   0:54
14         天安门东     5:21  23:23   0:24   0:05  1:06  5:12  23:48   0:52
15          王府井     5:23  23:25   0:26   0:07  1:08  5:10  23:46   0:50
16           东单     5:25  23:27   0:28   0:09  1:10  5:08  23:44   0:48
17          建国门     5:27  23:29   0:30   0:11  1:12  5:06  23:42   0:46
18          永安里     5:30  23:32   0:33   0:14  1:15  5:03  23:39   0:43
19           国贸     5:32  23:34   0:35   0:16  1:17  5:01  23:37   0:41
20          大望路     5:34  23:36   0:37   0:18  1:19  4:59  23:35   0:39
21           四惠     5:37  23:39   0:40   0:21  1:22  4:56  23:32   0:36
22          四惠东     5:40  23:42   0:43     ――    ――  5:04  23:29   0:33
23          高碑店     5:43  23:45   0:46     ――    ――  5:40  23:26   0:30
24         传媒大学     5:46  23:48   0:49     ――    ――  5:38  23:24   0:28
25           双桥     5:48  23:50   0:51     ――    ――  5:35  23:21   0:25
26           管庄     5:51  23:53   0:54     ――    ――  5:32  23:18   0:22
27          八里桥     5:54  23:56   0:57     ――    ――  5:29  23:15   0:19
28         通州北苑     5:57  23:59   1:00     ――    ――  5:26  23:12   0:16
29           果园     6:00   0:02   1:03     ――    ――  5:24  23:10   0:14
30          九棵树     6:02   0:04   1:05     ――    ――  5:22  23:08   0:12
31           梨园     6:04   0:06   1:07     ――    ――  5:19  23:05   0:09
32          临河里     6:06   0:08   1:09     ――    ――  5:17  23:03   0:07
33           土桥     4:59   0:10   1:11     ――    ――  5:15  23:01   0:05
34           花庄     5:02   0:13   1:14     ――    ――  5:12  22:58   0:02
35        环球度假区       ――     ――     ――     ――    ――  5:09  22:55  23:59

In [27]:
df[0].columns【0

'1号线八通线首末车时刻表'

In [34]:
k = 0
for i in  range(len(df)):
    print(i)
    for j in  range(len(df[i])):
        flag = True
        for item in metro_station.find({'n':df[i].iloc[j][0]}):
            k+=1
            item['ftime'] = df[i].iloc[j][1]
            item['ltime'] = df[i].iloc[j][2]
            if i ==  10:
                item['ltime'] = df[i].iloc[j][3]
                
#             print(item['ftime'],item['ltime'])
            metro_station.update({'_id':item['_id']},item,upsert=True)
            flag = False
        if flag:
            print(df[i].columns[0][0],df[i].iloc[j][0])
print(k)

0
1
2


D:\Anaconda3\envs\tensor_env\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  


3
4
6号线首末车时刻表 通运门
5
6
7
8
9
10
11
14号线首末车时刻表 陶然桥
14号线首末车时刻表 高家园
12
13
14
19号线首末车时刻表 太平桥
19号线首末车时刻表 北太平庄
15
16
17
18
19
20
21
22
469


In [77]:
df

[   1号线路网车站站间距信息                                                   
            起始站名   目的站名 距离（米） Unnamed: 3_level_1 Unnamed: 4_level_1
 0            古城  八角游乐园  1921                NaN                NaN
 1         八角游乐园    八宝山  1953                NaN                NaN
 2           八宝山    玉泉路  1479                NaN                NaN
 3           玉泉路    五棵松  1810                NaN                NaN
 4           五棵松    万寿路  1778                NaN                NaN
 5           万寿路    公主坟  1313                NaN                NaN
 6           公主坟  军事博物馆  1172                NaN                NaN
 7         军事博物馆    木樨地  1166                NaN                NaN
 8           木樨地   南礼士路  1291                NaN                NaN
 9          南礼士路    复兴门   424                NaN                NaN
 10          复兴门     西单  1596                NaN                NaN
 11           西单   天安门西  1217                NaN                NaN
 12         天安门西   天安门东   925                NaN

In [78]:

adjoin = db['adjoin']


In [73]:
import hashlib


'd41d8cd98f00b204e9800998ecf8427e'

In [85]:
fs = open('adjoin_rela.csv', 'w', encoding='utf_8_sig', newline='')
csv_w = csv.writer(fs)
col = [
   'id', 'pid1','pid2','n1','n2','distance','ln'
]
csv_w.writerow(col)
j = 1 
for i in range(len(df)):
    lname = df[i].columns[0][0].split('路网')[0]
    print(i)
    d = 0
    flag= True
    for k in range(len(df[i])):

        n1 = df[i].iloc[k][0]
        n2 = df[i].iloc[k][1]
        distance = d + int(df[i].iloc[k][2])
        pid1 = '-'
        pid2 = '-'
        for item in metro_station.find({'n': n1}):
            pid1 = item['_id']
        for item in metro_station.find({'n': n2}):
            pid2 = item['_id']
        if pid2 == '-':
            d = distance
            flag = False
        else:
            fn2 = n2
            fpid2 = pid2
            
        if pid1 == '-':
            d = 0
            flag = True
        else:
            fn1 = n1
            fpid1 = pid1
        a = hashlib.md5(("%s-%s"%(fpid1,fpid2)).encode('utf-8'))
        md5 = a.hexdigest()
        if flag:        
            item = {}
            item['_id'] = j
            item['pid1'] = fpid1
            item['pid2'] = fpid2
            item['n1'] = fn1
            item['n2'] = fn2
            item['dist'] = distance
            item['ln'] = lname
            j += 1
            csv_w.writerow(list(item.values()))
#             adjoin.update({'_id': item['_id']}, item, upsert=True)
            
            
fs.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [82]:
list(item.values())

['310bb3d4f09c21802dd33ed712884fa4',
 'BV10009415',
 'BV10000199',
 '3号航站楼',
 '三元桥',
 18322,
 '首都机场线']

In [59]:
df[11]

14号线路网车站站间距信息                                                   
            起始站名   目的站名 距离（米） Unnamed: 3_level_1 Unnamed: 4_level_1
0            张郭庄    园博园  1345                NaN                NaN
2            园博园    大瓦窑  4073                NaN                NaN
4            大瓦窑    郭庄子  1236                NaN                NaN
6            郭庄子     大井  2044                NaN                NaN
8             大井    七里庄  1579                NaN                NaN
10           七里庄     西局   845                NaN                NaN
12          北京南站    陶然桥   887                NaN                NaN
14           陶然桥   永定门外  1063                NaN                NaN
16          永定门外     景泰  1119                NaN                NaN
18            景泰    蒲黄榆  1025                NaN                NaN
20           蒲黄榆     方庄  1486                NaN                NaN
22            方庄    十里河  1618                NaN                NaN
24           十里河   南八里庄  1147                NaN                NaN
26          南八里庄  北工大西门  1276                NaN                NaN
28         北工大西门    平乐园  1128                NaN                NaN
30           平乐园    九龙山   897                NaN                NaN
32           九龙山    大望路  1780                NaN                NaN
34           大望路     红庙   708                NaN                NaN
36            红庙    金台路   894                NaN                NaN
38           金台路   朝阳公园  1085                NaN                NaN
40          朝阳公园     枣营  1221                NaN                NaN
42            枣营   东风北桥  2173                NaN                NaN
44          东风北桥     将台  1600                NaN                NaN
46            将台    高家园  1171                NaN                NaN
48           高家园    望京南   676                NaN                NaN
50           望京南     阜通  1168                NaN                NaN
52            阜通     望京   903                NaN                NaN
54            望京    东湖渠  1283                NaN                NaN
56           东湖渠    来广营  1100                NaN                NaN
58           来广营    善各庄  1364                NaN                NaN

In [4]:
ids = []
k = 1
fs = open('adjoin_rela_v2.csv', 'w', encoding='utf_8_sig', newline='')
csv_w = csv.writer(fs)
col = [
    'pid1','no','pid2','id','distance',
]
csv_w.writerow(col)



for item in metro_line.find():
    st_l = item['st']
    for i in range(len(st_l)-1):
        csv_w.writerow([st_l[i]['poiid'],str(i+1),st_l[i + 1]['poiid'],str(k),])
        k+=1
        
fs.close()
